# Yelp Dataset Recommendation

### Librerías

In [1]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')
import re
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

### Lectura del dataset

In [2]:
data = []
with open("yelp_academic_dataset_business.json", encoding = "utf-8") as data_file:
    for line in data_file:
        data.append(json.loads(line))

yelp_full = pd.DataFrame(data)
print("Total businesses:", yelp_full.shape[0])
yelp_full.head()

Total businesses: 150346


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."


### Selección de los restaurantes

In [3]:
Categories={}
yelp_full = yelp_full.dropna(subset=["categories"])
yelp_full["categories_splitted"] = yelp_full["categories"].apply(lambda x: [cat.strip() for cat in x.split(",")])
yelp_restaurants = yelp_full[yelp_full["categories_splitted"].apply(lambda cats: "Restaurants" in cats)]
yelp_restaurants = yelp_restaurants[~yelp_restaurants["categories_splitted"].apply(lambda cats: "Hair Salons" in cats)]
yelp_restaurants = yelp_restaurants[~yelp_restaurants["categories_splitted"].apply(lambda cats: "Beauty & Spas" in cats)]

In [4]:
print("Total Restaurants:", yelp_restaurants.shape[0])
yelp_restaurants.head()

Total Restaurants: 52182


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,categories_splitted
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...","[Restaurants, Food, Bubble Tea, Coffee & Tea, ..."
5,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,37015,36.269593,-87.058943,2.0,6,1,"{'BusinessParking': 'None', 'BusinessAcceptsCr...","Burgers, Fast Food, Sandwiches, Food, Ice Crea...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-22:0', '...","[Burgers, Fast Food, Sandwiches, Food, Ice Cre..."
8,k0hlBqXX-Bt0vf1op7Jr1w,Tsevi's Pub And Grill,8025 Mackenzie Rd,Affton,MO,63123,38.565165,-90.321087,3.0,19,0,"{'Caters': 'True', 'Alcohol': 'u'full_bar'', '...","Pubs, Restaurants, Italian, Bars, American (Tr...",None,"[Pubs, Restaurants, Italian, Bars, American (T..."
9,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,2312 Dickerson Pike,Nashville,TN,37207,36.208102,-86.768170,1.5,10,1,"{'RestaurantsAttire': ''casual'', 'Restaurants...","Ice Cream & Frozen Yogurt, Fast Food, Burgers,...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-21:0', '...","[Ice Cream & Frozen Yogurt, Fast Food, Burgers..."
11,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,,Tampa Bay,FL,33602,27.955269,-82.456320,4.0,10,1,"{'Alcohol': ''none'', 'OutdoorSeating': 'None'...","Vietnamese, Food, Restaurants, Food Trucks","{'Monday': '11:0-14:0', 'Tuesday': '11:0-14:0'...","[Vietnamese, Food, Restaurants, Food Trucks]"


### Incorporación reviews

In [5]:
data = []
with open("yelp_academic_dataset_review.json", encoding = "utf-8") as data_file:
    for line in data_file:
        data.append(json.loads(line))
yelp_reviews = pd.DataFrame(data)

print("Total reviews:", yelp_reviews.shape[0])
yelp_reviews.head()

Total reviews: 6990280


,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5.0,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3.0,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4.0,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15


In [6]:
yelp_reviews_grouped = yelp_reviews.groupby("business_id")["text"].apply(list).reset_index()
yelp = pd.merge(yelp_restaurants, yelp_reviews_grouped, on="business_id", how="left")
print("Total Restaurants:", yelp.shape[0])
yelp.head()

Total Restaurants: 52182


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,categories_splitted,text
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...","[Restaurants, Food, Bubble Tea, Coffee & Tea, ...",[This is nice little Chinese bakery in the hea...
1,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,37015,36.269593,-87.058943,2.0,6,1,"{'BusinessParking': 'None', 'BusinessAcceptsCr...","Burgers, Fast Food, Sandwiches, Food, Ice Crea...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-22:0', '...","[Burgers, Fast Food, Sandwiches, Food, Ice Cre...",[I have always liked Sonic as it is good fresh...
2,k0hlBqXX-Bt0vf1op7Jr1w,Tsevi's Pub And Grill,8025 Mackenzie Rd,Affton,MO,63123,38.565165,-90.321087,3.0,19,0,"{'Caters': 'True', 'Alcohol': 'u'full_bar'', '...","Pubs, Restaurants, Italian, Bars, American (Tr...",None,"[Pubs, Restaurants, Italian, Bars, American (T...","[I like Tsevis because they have good gyros, w..."
3,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,2312 Dickerson Pike,Nashville,TN,37207,36.208102,-86.768170,1.5,10,1,"{'RestaurantsAttire': ''casual'', 'Restaurants...","Ice Cream & Frozen Yogurt, Fast Food, Burgers,...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-21:0', '...","[Ice Cream & Frozen Yogurt, Fast Food, Burgers...",[Waited several minutes waiting to order. I wa...
4,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,,Tampa Bay,FL,33602,27.955269,-82.456320,4.0,10,1,"{'Alcohol': ''none'', 'OutdoorSeating': 'None'...","Vietnamese, Food, Restaurants, Food Trucks","{'Monday': '11:0-14:0', 'Tuesday': '11:0-14:0'...","[Vietnamese, Food, Restaurants, Food Trucks]",[I eat pho about 4 times a week and from a spe...


### Preprocesamiento

In [7]:
del yelp_full
del yelp_restaurants
del yelp_reviews

#### Eliminar duplicados

In [8]:
print("IDs duplicados:", yelp.business_id.duplicated().sum())
print("Restaurantes duplicados (nombre, dirección, ciudad y estado)", yelp.loc[: , ["name", "address", "city", "state"]].duplicated().sum())

IDs duplicados: 0
Restaurantes duplicados (nombre, dirección, ciudad y estado) 43


In [9]:
yelp.drop_duplicates(subset = ["name", "address", "city", "state"], inplace=True)
print("Total Restaurants after deleting duplicates:", yelp.shape[0])
yelp.head()

Total Restaurants after deleting duplicates: 52139


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,categories_splitted,text
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...","[Restaurants, Food, Bubble Tea, Coffee & Tea, ...",[This is nice little Chinese bakery in the hea...
1,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,37015,36.269593,-87.058943,2.0,6,1,"{'BusinessParking': 'None', 'BusinessAcceptsCr...","Burgers, Fast Food, Sandwiches, Food, Ice Crea...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-22:0', '...","[Burgers, Fast Food, Sandwiches, Food, Ice Cre...",[I have always liked Sonic as it is good fresh...
2,k0hlBqXX-Bt0vf1op7Jr1w,Tsevi's Pub And Grill,8025 Mackenzie Rd,Affton,MO,63123,38.565165,-90.321087,3.0,19,0,"{'Caters': 'True', 'Alcohol': 'u'full_bar'', '...","Pubs, Restaurants, Italian, Bars, American (Tr...",None,"[Pubs, Restaurants, Italian, Bars, American (T...","[I like Tsevis because they have good gyros, w..."
3,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,2312 Dickerson Pike,Nashville,TN,37207,36.208102,-86.768170,1.5,10,1,"{'RestaurantsAttire': ''casual'', 'Restaurants...","Ice Cream & Frozen Yogurt, Fast Food, Burgers,...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-21:0', '...","[Ice Cream & Frozen Yogurt, Fast Food, Burgers...",[Waited several minutes waiting to order. I wa...
4,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,,Tampa Bay,FL,33602,27.955269,-82.456320,4.0,10,1,"{'Alcohol': ''none'', 'OutdoorSeating': 'None'...","Vietnamese, Food, Restaurants, Food Trucks","{'Monday': '11:0-14:0', 'Tuesday': '11:0-14:0'...","[Vietnamese, Food, Restaurants, Food Trucks]",[I eat pho about 4 times a week and from a spe...


### Limpieza columna Attributes

In [10]:
import pandas as pd
import ast

def limpiar_valor(valor):
    # Si es string y parece diccionario, parsear
    if isinstance(valor, str):
        try:
            posible_dict = ast.literal_eval(valor)
            if isinstance(posible_dict, dict):
                return limpiar_diccionario(posible_dict)
        except (ValueError, SyntaxError):
            pass
        
        # Limpiar strings tipo u'valor' o 'valor'
        if valor.startswith("u'") and valor.endswith("'"):
            valor = valor[2:-1]
        elif valor.startswith("'") and valor.endswith("'"):
            valor = valor[1:-1]
        
        # Pasar a minúscula y capitalizar (opcional)
        valor = valor.lower().capitalize()
    
    return valor

def limpiar_diccionario(dic):
    dic_limpio = {}
    for k, v in dic.items():
        if isinstance(v, dict):
            dic_limpio[k] = limpiar_diccionario(v)
        else:
            dic_limpio[k] = limpiar_valor(v)
    return dic_limpio

# Supongamos que tu DataFrame es df y la columna se llama 'attributes'
yelp['attributes_clean'] = yelp['attributes'].apply(lambda x: limpiar_diccionario(x) if isinstance(x, dict) else x)

In [11]:
from collections import defaultdict

valores_atributos = defaultdict(set)

def recolectar_valores(dic, prefijo=''):
    for clave, valor in dic.items():
        clave_completa = f"{prefijo}.{clave}" if prefijo else clave
        if isinstance(valor, dict):
            recolectar_valores(valor, prefijo=clave_completa)
        else:
            valores_atributos[clave_completa].add(valor)

for dic in yelp['attributes_clean']:
    if isinstance(dic, dict):
        recolectar_valores(dic)

# Convertimos a dict normal si quieres visualizarlo mejor
valores_atributos = dict(valores_atributos)

# Ejemplo de visualización
for clave, valores in valores_atributos.items():
    print(f"{clave}: {valores}")

RestaurantsDelivery: {'None', 'False', 'True'}
OutdoorSeating: {'None', 'False', 'True'}
BusinessAcceptsCreditCards: {'None', 'False', 'True'}
BusinessParking.garage: {False, True, None}
BusinessParking.street: {False, True, None}
BusinessParking.validated: {False, True, None}
BusinessParking.lot: {False, True, None}
BusinessParking.valet: {False, True}
BikeParking: {'None', 'False', 'True'}
RestaurantsPriceRange2: {'3', '1', '4', '2', 'None'}
RestaurantsTakeOut: {'False', 'None', 'True'}
ByAppointmentOnly: {'None', 'False', 'True'}
WiFi: {'No', 'Paid', 'None', 'Free'}
Alcohol: {'Beer_and_wine', 'None', 'Full_bar'}
Caters: {'None', 'False', 'True'}
BusinessParking: {'None'}
RestaurantsAttire: {'Dressy', 'Casual', 'Formal', 'None'}
RestaurantsReservations: {'None', 'False', 'True'}
Ambience: {'None'}
GoodForKids: {'None', 'False', 'True'}
CoatCheck: {'None', 'False', 'True'}
DogsAllowed: {'None', 'False', 'True'}
RestaurantsTableService: {'None', 'False', 'True'}
RestaurantsGoodForGroup

### Unir reseñas en único texto

In [12]:
yelp["text_to_bert"] = yelp["text"].apply(lambda x: " ".join(x) if isinstance(x, list) else x)

In [13]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(yelp["text_to_bert"].tolist(), show_progress_bar=True)

Batches: 100%|██████████| 1630/1630 [09:51<00:00,  2.76it/s]


In [15]:
from sklearn.metrics.pairwise import cosine_similarity

query = "Hi, whats'up i want to eat some chicken wings like KFC"
query_embedding = model.encode([query])

cosine_similarities = cosine_similarity(query_embedding, embeddings)[0]
top_n = cosine_similarities.argsort()[-10:][::-1]

res = yelp.iloc[top_n][["name", "address", "city", "state", "stars", "categories"]]
res

,name,address,city,state,stars,categories
20232,Tasty Shop,2327 E Hillsborough Ave,Tampa,FL,5.0,"Restaurants, Seafood, Chicken Wings"
44832,KFC,322 W Main St,Hendersonville,TN,2.0,"Restaurants, Chicken Shop, Chicken Wings, Fast..."
9802,Chicken Shack Express,782 Nissan Dr,Smyrna,TN,4.0,"Restaurants, Chicken Wings, Food Trucks, Food"
4703,Wing Shop,408 Doylestown Rd,Montgomeryville,PA,3.0,"Restaurants, Chicken Wings"
42438,KFC,65 Damonte Ranch Pkwy,Reno,NV,2.0,"Chicken Shop, Chicken Wings, Restaurants, Fast..."
11632,Wingstop,4912 S Emerson Ave,Indianapolis,IN,2.5,"Restaurants, Chicken Wings"
3747,BBQ Chicken,7034 Terminal Square,Upper Darby,PA,4.5,"Restaurants, Chicken Shop, Bubble Tea, Chicken..."
37413,The Sunday House,1020 Industrial Dr,Pleasant View,TN,3.5,"Sandwiches, Waffles, Breakfast & Brunch, Soul ..."
42262,KFC,4528 W 38th St,Indianapolis,IN,1.5,"Fast Food, Chicken Shop, Chicken Wings, Restau..."
16129,KFC,336 Harding Pl,Nashville,TN,1.5,"Restaurants, Chicken Shop, Fast Food, Chicken ..."


In [16]:
yelp.attributes.sample(1).values[0]

{'GoodForKids': 'True',
 'RestaurantsReservations': 'False',
 'Alcohol': "u'full_bar'",
 'RestaurantsPriceRange2': '2',
 'HasTV': 'True',
 'RestaurantsAttire': "u'casual'",
 'BikeParking': 'True',
 'WiFi': "'free'",
 'RestaurantsGoodForGroups': 'True',
 'Caters': 'False',
 'RestaurantsTakeOut': 'True',
 'OutdoorSeating': 'True',
 'NoiseLevel': "'average'",
 'BusinessAcceptsCreditCards': 'True',
 'GoodForMeal': "{'dessert': True, 'latenight': False, 'lunch': True, 'dinner': True, 'brunch': False, 'breakfast': False}",
 'Ambience': "{'touristy': None, 'hipster': False, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': None, 'upscale': False, 'classy': False, 'casual': True}",
 'RestaurantsTableService': 'True',
 'BusinessParking': "{'garage': False, 'street': None, 'validated': False, 'lot': True, 'valet': False}",
 'RestaurantsDelivery': 'True'}

Good for kids | Do not accept reservations | Has any type of alcohol | Medium price | Has TV | Cassual attire | Mexican | Coffe & Tea